# Transformer Model  
Predicting Droughts Using Weather & Soil Data in the US

## Instructions  
This notebook is designed to run on Google Colab. It requires **at least 25 GB of system RAM** and **1.5 GB of GPU RAM**, so a **T4 GPU with high-RAM runtime** is recommended.

You do not need to manually install any datasets. Simply add your Kaggle credentials in the designated code cell, and the data will be downloaded and saved to your Google Drive.

If you plan to run this notebook on another machine (e.g., your local computer), you can skip the `drive.mount` step and change the data directory path to your local destination. Everything else should work as expected.


## Install Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# change your kaggle information here
os.environ['KAGGLE_USERNAME'] = "YourKaggleUsername"
os.environ['KAGGLE_KEY'] = "YourKaggleKey"

DATA_DIR = "/content/drive/MyDrive/us_drought_data"  # folder in Google Drive
ZIP_PATH = os.path.join(DATA_DIR, "us-drought-meteorological-data.zip")

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR, exist_ok=True)

soil_csv  = os.path.join(DATA_DIR, "soil_data.csv")
train_csv = os.path.join(DATA_DIR, "train_timeseries/train_timeseries.csv")
val_csv   = os.path.join(DATA_DIR, "validation_timeseries/validation_timeseries.csv")
test_csv  = os.path.join(DATA_DIR, "test_timeseries/test_timeseries.csv")


if not (os.path.isfile(soil_csv) and
        os.path.isfile(train_csv) and
        os.path.isfile(val_csv) and
        os.path.isfile(test_csv)):
    !kaggle datasets download -d cdminix/us-drought-meteorological-data -p {DATA_DIR} -o
    !unzip -q {ZIP_PATH} -d {DATA_DIR}
    print("\nDataset downloaded and unzipped to Google Drive.")

else:
    print("\nDataset already found in Google Drive. Skipping download.")



Dataset already found in Google Drive. Skipping download.


## Import

In [3]:
import os, math, warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.swa_utils import AveragedModel, update_bn

warnings.filterwarnings("ignore")

## Preprocessing

In [4]:
# dynamic feature selection: drops any feature if it is no longer useful as an indicator
weather_features = [
    "PRECTOT","PS","QV2M","T2M","T2MDEW","T2MWET","T2M_MAX","T2M_MIN",
    "T2M_RANGE","TS","WS10M","WS10M_MAX","WS10M_MIN","WS10M_RANGE",
    "WS50M","WS50M_MAX","WS50M_MIN","WS50M_RANGE"
]
soil_features = [
    "lat","lon","elevation","slope1","slope2","slope3","slope4","slope5",
    "slope6","slope7","slope8","aspectN","aspectE","aspectS","aspectW",
    "aspectUnknown","WAT_LAND","NVG_LAND","URB_LAND","GRS_LAND","FOR_LAND",
    "CULTRF_LAND","CULTIR_LAND","CULT_LAND","SQ1","SQ2","SQ3","SQ4","SQ5",
    "SQ6","SQ7"
]

# fill nan values with smoothly interpolated values
# e.g [0, nan, nan, nan, 1] -> [0, 0.25, 0.5, 0.75, 1]
def interpolate_group(df, cols):
    arr = df[cols].values
    idx = np.arange(len(df))
    for j in range(arr.shape[1]):
        col = arr[:, j]
        mask = ~np.isnan(col)
        if mask.any():
            arr[:, j] = np.interp(idx, idx[mask], col[mask])
    df[cols] = arr
    return df

# compute the median and iqr on train dataset
def robust_fit(values: np.ndarray):
    median = np.nanmedian(values)
    q75 = np.nanpercentile(values, 75)
    q25 = np.nanpercentile(values, 25)
    iqr = max(q75 - q25, 1e-6)
    return median, iqr

# apply to train, test, and val datasets to reduce the impact of outliers
def robust_transform(values: np.ndarray, median: float, iqr: float):
    return (values - median) / iqr

def load_datasets(data_dir):
    soil  = pd.read_csv(os.path.join(data_dir, "soil_data.csv"))
    train = pd.read_csv(os.path.join(data_dir, "train_timeseries/train_timeseries.csv"))
    val   = pd.read_csv(os.path.join(data_dir, "validation_timeseries/validation_timeseries.csv"))
    test  = pd.read_csv(os.path.join(data_dir, "test_timeseries/test_timeseries.csv"))

    for df in (train, val, test):
        df["date"] = pd.to_datetime(df["date"])
        df.sort_values(["fips","date"], inplace=True)
        doy = df["date"].dt.dayofyear
        df["season_sin"] = np.sin(2*np.pi*(doy-1)/365)
        df["season_cos"] = np.cos(2*np.pi*(doy-1)/365)
        df["score_past"] = df.groupby("fips")["score"].shift(7)

    # merge soil data (static) with time series data
    train = train.merge(soil, on="fips", how="left")
    val   = val.merge(soil,   on="fips", how="left")
    test  = test.merge(soil,  on="fips", how="left")

    # interpolate weather + score_past
    interp_cols = weather_features + ["score_past"]
    train = train.groupby("fips", group_keys=False).apply(interpolate_group, interp_cols)
    val   = val.  groupby("fips", group_keys=False).apply(interpolate_group, interp_cols)
    test  = test. groupby("fips", group_keys=False).apply(interpolate_group, interp_cols)

    # fill static nan with train mean
    for feat in soil_features:
        mean_val = train[feat].mean()
        for df in (train, val, test):
            df[feat].fillna(mean_val, inplace=True)

    # robust scaling & some feature engineering
    dyn_feats = weather_features + ["season_sin","season_cos","score_past"]
    stat_feats = soil_features

    # fit scale params on train only
    params_dyn  = {c: robust_fit(train[c].values) for c in dyn_feats}
    params_stat = {c: robust_fit(train[c].values) for c in stat_feats}

    for df in (train, val, test):
        for c in dyn_feats:
            med, iqr = params_dyn[c]
            df[c] = robust_transform(df[c].values, med, iqr)
        for c in stat_feats:
            med, iqr = params_stat[c]
            df[c] = robust_transform(df[c].values, med, iqr)

    return train, val, test, dyn_feats, stat_feats

In [ ]:
class DroughtDataset(Dataset):
    def __init__(self, df, dyn_feats, sta_feats, seq_len=240, horizon=6):
        self.seq_len, self.horizon = seq_len, horizon
        self.dyn, self.sta = dyn_feats, sta_feats
        self.data, self.indices = {}, []

        df = df.sort_values(["fips","date"])
        for fips, g in df.groupby("fips"):
            Xd = g[self.dyn].values.astype(np.float32)
            Xs = g[self.sta].iloc[0].values.astype(np.float32)
            y  = g["score"].values
            self.data[fips] = {"Xd":Xd, "Xs":Xs, "y":y}

            max_start = len(g) - seq_len - 7*horizon + 1
            for i in range(max_start):
                idxs = [i+seq_len+7*w for w in range(horizon)]
                if not np.isnan(y[idxs]).any():
                    self.indices.append((fips, i))

    def __len__(self): return len(self.indices)

    def __getitem__(self, idx):
        fips, start = self.indices[idx]
        d = self.data[fips]
        X_dyn = torch.from_numpy(d["Xd"][start:start+self.seq_len])
        X_sta = torch.from_numpy(d["Xs"])
        y_idx = [start+self.seq_len+7*w for w in range(self.horizon)]
        y = torch.from_numpy(d["y"][y_idx].astype(np.float32))
        return X_dyn, X_sta, y

## Training

In [5]:
# hyper parameter and constant here
# play with different hyper parameter to get the best result
EPOCHS       = 10
BATCH        = 2048
SEQ_LEN      = 7
HORIZON      = 6
HIDDEN       = 256
LAYERS       = 3
HEADS        = 8
DROPOUT      = 0.5
LR           = 3e-4
GRAD_CLIP    = 1.0
USE_SWA      = True
SWA_START_EP = 4
CKPT_PATH    = "best_transformer_mae.pt"
DEVICE       = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class DroughtTransformer(nn.Module):
    def __init__(self, dyn_in, sta_in, hidden, layers, heads,
                 seq_len, horizon, dropout=0.4):
        super().__init__()
        self.embed = nn.Linear(dyn_in, hidden) # dynamic, time series data
        self.register_buffer("pos", self._positional_encoding(seq_len, hidden))
        enc_layer = nn.TransformerEncoderLayer(
            hidden, heads, dim_feedforward=hidden*4,
            dropout=dropout, batch_first=True)
        self.enc = nn.TransformerEncoder(enc_layer, layers)
        self.fc_sta = nn.Sequential(
            nn.LayerNorm(sta_in),
            nn.Linear(sta_in, hidden),
            nn.ReLU()
        ) # soil data
        self.fc_out = nn.Sequential(
            nn.Linear(hidden*2, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, horizon)
        )

    @staticmethod
    def _positional_encoding(seq_len, d_model):
        pos = torch.arange(seq_len, dtype=torch.float32).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2).float() *
                        -(math.log(10000.0) / d_model))
        pe  = torch.zeros(seq_len, d_model)
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        return pe.unsqueeze(0)        # (1, seq_len, d_model)

    def forward(self, x_dyn, x_sta):
        x_dyn = self.embed(x_dyn) + self.pos[:, :x_dyn.size(1)]
        h_dyn = self.enc(x_dyn)[:, -1]
        h_sta = self.fc_sta(x_sta)
        h = torch.cat([h_dyn, h_sta], 1)
        return self.fc_out(h)

In [ ]:
def train(model, train_ld, val_ld):
    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
    sched = OneCycleLR(opt, max_lr=LR*10, epochs=EPOCHS,
                       steps_per_epoch=len(train_ld),
                       pct_start=0.3, anneal_strategy="cos",
                       div_factor=10, final_div_factor=1e4)

    scaler = GradScaler()
    best_mae = float("inf"); bad=0; patience=3
    swa_model = AveragedModel(model) if USE_SWA else None

    for ep in range(1, EPOCHS+1):
        model.train()
        pbar = tqdm(train_ld, desc=f"[{ep}/{EPOCHS}]")
        for Xd, Xs, y in pbar:
            Xd, Xs, y = Xd.to(DEVICE), Xs.to(DEVICE), y.to(DEVICE)
            opt.zero_grad(set_to_none=True)
            with autocast():
                pred = model(Xd, Xs)
                loss = F.l1_loss(pred, y)
            scaler.scale(loss).backward()
            scaler.unscale_(opt)
            nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
            scaler.step(opt); scaler.update(); sched.step()
            pbar.set_postfix(mae=f"{loss.item():.4f}")

        # validation
        model.eval();
        mae_sum = 0.0;
        n = 0
        with torch.no_grad(), autocast():
            for Xd, Xs, y in val_ld:
                Xd, Xs, y = Xd.to(DEVICE), Xs.to(DEVICE), y.to(DEVICE)
                pred = model(Xd, Xs)
                mae_sum += F.l1_loss(pred, y, reduction='sum').item()
                n += y.numel()
        val_mae = mae_sum / n
        print(f"  ↳ val MAE={val_mae:.4f}")

        if val_mae < best_mae:
            best_mae = val_mae; bad=0
            torch.save({"model":model.state_dict()}, CKPT_PATH)
            print("    ✓ best saved")
        else:
            bad += 1
            if bad >= patience:
                print("    ✕ early stop"); break

        if USE_SWA and ep >= SWA_START_EP:
            swa_model.update_parameters(model)

    # finalize SWA
    if USE_SWA and swa_model is not None and ep >= SWA_START_EP:
        update_bn(train_ld, swa_model, device=DEVICE)
        torch.save({"model":swa_model.state_dict()}, "best_swa.pt")

## Evaluation and run

In [6]:
def evaluate(model, dl):
    model.eval();
    mae_sum = 0.0;
    n = 0
    with torch.no_grad(), autocast():
        for Xd, Xs, y in tqdm(dl, desc="[test]"):
            Xd, Xs, y = Xd.to(DEVICE), Xs.to(DEVICE), y.to(DEVICE)
            pred = model(Xd, Xs)
            mae_sum += F.l1_loss(pred, y, reduction='sum').item()
            n += y.numel()
    return mae_sum / n

In [7]:
print("Loading & preprocessing …")
train_df, val_df, test_df, dyn_feats, sta_feats = load_datasets(DATA_DIR)

train_ds = DroughtDataset(train_df, dyn_feats, sta_feats, SEQ_LEN, HORIZON)
val_ds   = DroughtDataset(val_df,   dyn_feats, sta_feats, SEQ_LEN, HORIZON)
test_ds  = DroughtDataset(test_df,  dyn_feats, sta_feats, SEQ_LEN, HORIZON)

train_ld = DataLoader(train_ds, BATCH, shuffle=True,  num_workers=2, pin_memory=True)
val_ld   = DataLoader(val_ds,   BATCH, shuffle=False, num_workers=2, pin_memory=True)
test_ld  = DataLoader(test_ds,  BATCH, shuffle=False, num_workers=2, pin_memory=True)

model = DroughtTransformer(
    dyn_in=len(dyn_feats), sta_in=len(sta_feats),
    hidden=HIDDEN, layers=LAYERS, heads=HEADS,
    seq_len=SEQ_LEN, horizon=HORIZON, dropout=DROPOUT
).to(DEVICE)

print("Training …")
train(model, train_ld, val_ld)

print("\nEvaluating best checkpoint …")
ckpt = torch.load(CKPT_PATH, map_location=DEVICE)
model.load_state_dict(ckpt["model"])
test_mae = evaluate(model, test_ld)
print(f"Test MAE = {test_mae:.4f}")

Loading & preprocessing …
Training …


[1/10]: 100%|██████████| 1336/1336 [01:04<00:00, 20.62it/s, mae=0.2725]


  ↳ val MAE=0.2693
    ✓ best saved


[2/10]: 100%|██████████| 1336/1336 [01:05<00:00, 20.32it/s, mae=0.3195]


  ↳ val MAE=0.2492
    ✓ best saved


...

  ↳ val MAE=0.2883


[5/10]: 100%|██████████| 1336/1336 [01:05<00:00, 20.32it/s, mae=0.2998]


  ↳ val MAE=0.2773
    ✕ early stop

Evaluating best checkpoint …


[test]: 100%|██████████| 149/149 [00:03<00:00, 45.05it/s]

Test MAE = 0.1982
